# LSA con Comentarios

Acá hago todo el modelo que se basa en usar LSA para poder encontrar los juegos más parecidos en base a la similitud de párrafos (comentarios de un juego en particular concatenados). Let's hope for the best.

In [ ]:
import pandas as pd 
from google.colab import drive
drive.mount('/content/drive')

import matplotlib.pylab as plt
import numpy as np

Mounted at /content/drive


In [ ]:
# Importamos nltk para extraer stopwords 
import nltk 
nltk.download('stopwords')

# Librería para hacer wordclouds
from wordcloud import WordCloud

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Objetos de sklearn para hacer LSA
from sklearn.feature_extraction.text import CountVectorizer # Contador de frecuencia
from sklearn.feature_extraction.text import TfidfTransformer # Creador de tf-idf
from sklearn.decomposition import TruncatedSVD # Singular Value Decomposition

In [ ]:
filename1 = '/content/drive/My Drive/Colab Notebooks/Tp Final LDD/metacritic_game_info.csv'
filename2 = '/content/drive/My Drive/Colab Notebooks/Tp Final LDD/df_eng.csv'

In [ ]:
df_juegos = pd.read_csv(filename1)
df_comentarios = pd.read_csv(filename2)

df_juegos.dropna()
df_comentarios.dropna()

,Unnamed: 0,level_0,index,Unnamed: 0.1,Title,Platform,Userscore,Comment,Username
0,0,0,0,0,The Legend of Zelda: Ocarina of Time,Nintendo64,10,"Everything in OoT is so near at perfection, it...",SirCaestus
1,1,1,1,1,The Legend of Zelda: Ocarina of Time,Nintendo64,10,I won't bore you with what everyone is already...,Kaistlin
2,2,2,2,2,The Legend of Zelda: Ocarina of Time,Nintendo64,10,Anyone who gives the masterpiece below a 7 or ...,Jacody
3,3,3,3,3,The Legend of Zelda: Ocarina of Time,Nintendo64,10,I'm one of those people who think that this is...,doodlerman
4,4,4,4,4,The Legend of Zelda: Ocarina of Time,Nintendo64,10,This game is the highest rated game on Metacr...,StevenA
...,...,...,...,...,...,...,...,...,...
271439,271439,282977,283978,283978,Etrian Odyssey Untold: The Millennium Girl,3DS,7,"Extremely similar to EO:4, which obviously isn...",RileyWRussell
271440,271440,282978,283979,283979,Etrian Odyssey Untold: The Millennium Girl,3DS,0,Typical overrated Atlus trash. A game i should...,TemplarGR
271441,271441,282979,283980,283980,Etrian Odyssey Untold: The Millennium Girl,3DS,9,While I find the story mode to have annoying c...,midipon
271442,271442,282980,283981,283981,Etrian Odyssey Untold: The Millennium Girl,3DS,8,"Pretty good, but it certainly lacks the visual...",night4


In [ ]:
# 1 Player, Multiplayer, Online

def corregirPlayer(player):
  if 'No' in player or 'not' in player or player == '1 Player':
    return '1 Player'
  elif 'Online' in player:
    return 'Online'
  else:
    return 'Multiplayer'

In [ ]:
df_juegos['No_Players'] = df_juegos['No_Players'].astype(str)

In [ ]:
df_juegos['No_Players'] = df_juegos['No_Players'].apply(corregirPlayer)

In [ ]:
df_juegos #el binarizer no lo uso porque no lo necesito

,Unnamed: 0,Title,Year,Publisher,Genre,Platform,Metascore,Avg_Userscore,No_Players
0,0,The Legend of Zelda: Ocarina of Time,1998,Nintendo,Action Adventure;Fantasy,Nintendo64,99,9.1,1 Player
1,1,Tony Hawk's Pro Skater 2,2000,NeversoftEntertainment,Sports;Alternative;Skateboarding,PlayStation,98,7.4,Multiplayer
2,2,Grand Theft Auto IV,2008,RockstarNorth,Action Adventure;Modern;Modern;Open-World,PlayStation3,98,7.5,1 Player
3,3,SoulCalibur,1999,Namco,Action;Fighting;3D,Dreamcast,98,8.6,Multiplayer
4,4,Grand Theft Auto IV,2008,RockstarNorth,Action Adventure;Modern;Modern;Open-World,Xbox360,98,7.9,1 Player
...,...,...,...,...,...,...,...,...,...
4995,4995,Donut County,2018,BenEsposito,Action Adventure;General,PC,77,8.1,1 Player
4996,4996,MotorStorm: Apocalypse,2011,EvolutionStudios,Driving;Racing;Simulation;Rally / Offroad;Rall...,PlayStation3,77,7.7,Online
4997,4997,The Last Guy,2008,SCEJapanStudio,Action Adventure;Sci-Fi;Sci-Fi;General,PlayStation3,77,6.8,1 Player
4998,4998,Valiant Hearts: The Great War,2014,UbisoftMontpellier,Platformer;2D;Action;Platformer;2D,PlayStation4,77,8.4,1 Player


In [ ]:
df_comentarios

,Unnamed: 0,level_0,index,Unnamed: 0.1,Title,Platform,Userscore,Comment,Username
0,0,0,0,0,The Legend of Zelda: Ocarina of Time,Nintendo64,10,"Everything in OoT is so near at perfection, it...",SirCaestus
1,1,1,1,1,The Legend of Zelda: Ocarina of Time,Nintendo64,10,I won't bore you with what everyone is already...,Kaistlin
2,2,2,2,2,The Legend of Zelda: Ocarina of Time,Nintendo64,10,Anyone who gives the masterpiece below a 7 or ...,Jacody
3,3,3,3,3,The Legend of Zelda: Ocarina of Time,Nintendo64,10,I'm one of those people who think that this is...,doodlerman
4,4,4,4,4,The Legend of Zelda: Ocarina of Time,Nintendo64,10,This game is the highest rated game on Metacr...,StevenA
...,...,...,...,...,...,...,...,...,...
271439,271439,282977,283978,283978,Etrian Odyssey Untold: The Millennium Girl,3DS,7,"Extremely similar to EO:4, which obviously isn...",RileyWRussell
271440,271440,282978,283979,283979,Etrian Odyssey Untold: The Millennium Girl,3DS,0,Typical overrated Atlus trash. A game i should...,TemplarGR
271441,271441,282979,283980,283980,Etrian Odyssey Untold: The Millennium Girl,3DS,9,While I find the story mode to have annoying c...,midipon
271442,271442,282980,283981,283981,Etrian Odyssey Untold: The Millennium Girl,3DS,8,"Pretty good, but it certainly lacks the visual...",night4


In [ ]:
df_comentarios = df_comentarios[['Title','Platform','Userscore','Comment','Username']]

In [ ]:
df_comentarios = df_comentarios.drop_duplicates()
df_comentarios = df_comentarios.reset_index()

In [ ]:
df_comentarios

,index,Title,Platform,Userscore,Comment,Username
0,0,The Legend of Zelda: Ocarina of Time,Nintendo64,10,"Everything in OoT is so near at perfection, it...",SirCaestus
1,1,The Legend of Zelda: Ocarina of Time,Nintendo64,10,I won't bore you with what everyone is already...,Kaistlin
2,2,The Legend of Zelda: Ocarina of Time,Nintendo64,10,Anyone who gives the masterpiece below a 7 or ...,Jacody
3,3,The Legend of Zelda: Ocarina of Time,Nintendo64,10,I'm one of those people who think that this is...,doodlerman
4,4,The Legend of Zelda: Ocarina of Time,Nintendo64,10,This game is the highest rated game on Metacr...,StevenA
...,...,...,...,...,...,...
269980,271439,Etrian Odyssey Untold: The Millennium Girl,3DS,7,"Extremely similar to EO:4, which obviously isn...",RileyWRussell
269981,271440,Etrian Odyssey Untold: The Millennium Girl,3DS,0,Typical overrated Atlus trash. A game i should...,TemplarGR
269982,271441,Etrian Odyssey Untold: The Millennium Girl,3DS,9,While I find the story mode to have annoying c...,midipon
269983,271442,Etrian Odyssey Untold: The Millennium Girl,3DS,8,"Pretty good, but it certainly lacks the visual...",night4


In [ ]:
uniqueJuegos = df_juegos['Title'].unique()

In [ ]:
len(uniqueJuegos)

3438

In [ ]:
df_juegos = df_juegos.drop_duplicates(subset = ['Title'])
df_juegos = df_juegos.reset_index()

In [ ]:
df_juegos

,index,Unnamed: 0,Title,Year,Publisher,Genre,Platform,Metascore,Avg_Userscore,No_Players
0,0,0,The Legend of Zelda: Ocarina of Time,1998,Nintendo,Action Adventure;Fantasy,Nintendo64,99,9.1,1 Player
1,1,1,Tony Hawk's Pro Skater 2,2000,NeversoftEntertainment,Sports;Alternative;Skateboarding,PlayStation,98,7.4,Multiplayer
2,2,2,Grand Theft Auto IV,2008,RockstarNorth,Action Adventure;Modern;Modern;Open-World,PlayStation3,98,7.5,1 Player
3,3,3,SoulCalibur,1999,Namco,Action;Fighting;3D,Dreamcast,98,8.6,Multiplayer
4,5,5,Super Mario Galaxy,2007,Nintendo,Action;Platformer;Platformer;3D;3D,Wii,97,9.0,1 Player
...,...,...,...,...,...,...,...,...,...,...
3433,4990,4990,Element4l,2013,i-illusions,Action;Platformer;Platformer;2D;2D,PC,77,7.7,1 Player
3434,4991,4991,Anomaly 2,2013,11bitstudios,Action;Strategy;General;Real-Time;Sci-Fi;Sci-F...,PC,77,7.9,1 Player
3435,4995,4995,Donut County,2018,BenEsposito,Action Adventure;General,PC,77,8.1,1 Player
3436,4996,4996,MotorStorm: Apocalypse,2011,EvolutionStudios,Driving;Racing;Simulation;Rally / Offroad;Rall...,PlayStation3,77,7.7,Online


In [ ]:
df_juegos = df_juegos[['Title', 'Year', 'Publisher', 'Genre',
       'Platform', 'Metascore', 'Avg_Userscore', 'No_Players']]

In [ ]:
df_juegos

,Title,Year,Publisher,Genre,Platform,Metascore,Avg_Userscore,No_Players
0,The Legend of Zelda: Ocarina of Time,1998,Nintendo,Action Adventure;Fantasy,Nintendo64,99,9.1,1 Player
1,Tony Hawk's Pro Skater 2,2000,NeversoftEntertainment,Sports;Alternative;Skateboarding,PlayStation,98,7.4,Multiplayer
2,Grand Theft Auto IV,2008,RockstarNorth,Action Adventure;Modern;Modern;Open-World,PlayStation3,98,7.5,1 Player
3,SoulCalibur,1999,Namco,Action;Fighting;3D,Dreamcast,98,8.6,Multiplayer
4,Super Mario Galaxy,2007,Nintendo,Action;Platformer;Platformer;3D;3D,Wii,97,9.0,1 Player
...,...,...,...,...,...,...,...,...
3433,Element4l,2013,i-illusions,Action;Platformer;Platformer;2D;2D,PC,77,7.7,1 Player
3434,Anomaly 2,2013,11bitstudios,Action;Strategy;General;Real-Time;Sci-Fi;Sci-F...,PC,77,7.9,1 Player
3435,Donut County,2018,BenEsposito,Action Adventure;General,PC,77,8.1,1 Player
3436,MotorStorm: Apocalypse,2011,EvolutionStudios,Driving;Racing;Simulation;Rally / Offroad;Rall...,PlayStation3,77,7.7,Online


In [ ]:
def indJuegoSinComentario(dfCom, dfJug):
  ind = []
  for i in range(len(dfJug)):
    if not(dfJug['Title'][i] in dfCom['Title'].unique()):
      ind.append(i)
    else:
      continue
  return ind

In [ ]:
indSinCom = indJuegoSinComentario(df_comentarios, df_juegos) #tarda 60 secs + o -

In [ ]:
3438 - len(indSinCom) # juegos con comentarios

2324

In [ ]:
df_juegos = df_juegos.drop(df_juegos.index[indSinCom])

In [ ]:
df_juegos

,Title,Year,Publisher,Genre,Platform,Metascore,Avg_Userscore,No_Players
0,The Legend of Zelda: Ocarina of Time,1998,Nintendo,Action Adventure;Fantasy,Nintendo64,99,9.1,1 Player
1,Tony Hawk's Pro Skater 2,2000,NeversoftEntertainment,Sports;Alternative;Skateboarding,PlayStation,98,7.4,Multiplayer
2,Grand Theft Auto IV,2008,RockstarNorth,Action Adventure;Modern;Modern;Open-World,PlayStation3,98,7.5,1 Player
3,SoulCalibur,1999,Namco,Action;Fighting;3D,Dreamcast,98,8.6,Multiplayer
4,Super Mario Galaxy,2007,Nintendo,Action;Platformer;Platformer;3D;3D,Wii,97,9.0,1 Player
...,...,...,...,...,...,...,...,...
2362,N++: Ultimate Edition,2018,MetanetSoftwareInc.,Action;Platformer;2D,Switch,80,7.4,1 Player
2363,"Guns, Gore & Cannoli 2",2018,CrazyMonkeyStudios,Action;Shooter;Third-Person;Arcade,PC,80,7.3,Online
2364,Battle Brothers,2017,OverhypeStudios,Strategy;Turn-Based;Tactics,PC,80,7.5,1 Player
2365,BoxBoxBoy!,2016,HALLabs,Puzzle;Action,3DS,80,7.9,1 Player


In [ ]:
df_juegos = df_juegos.reset_index()

In [ ]:
df_juegos

,index,Title,Year,Publisher,Genre,Platform,Metascore,Avg_Userscore,No_Players
0,0,The Legend of Zelda: Ocarina of Time,1998,Nintendo,Action Adventure;Fantasy,Nintendo64,99,9.1,1 Player
1,1,Tony Hawk's Pro Skater 2,2000,NeversoftEntertainment,Sports;Alternative;Skateboarding,PlayStation,98,7.4,Multiplayer
2,2,Grand Theft Auto IV,2008,RockstarNorth,Action Adventure;Modern;Modern;Open-World,PlayStation3,98,7.5,1 Player
3,3,SoulCalibur,1999,Namco,Action;Fighting;3D,Dreamcast,98,8.6,Multiplayer
4,4,Super Mario Galaxy,2007,Nintendo,Action;Platformer;Platformer;3D;3D,Wii,97,9.0,1 Player
...,...,...,...,...,...,...,...,...,...
2319,2362,N++: Ultimate Edition,2018,MetanetSoftwareInc.,Action;Platformer;2D,Switch,80,7.4,1 Player
2320,2363,"Guns, Gore & Cannoli 2",2018,CrazyMonkeyStudios,Action;Shooter;Third-Person;Arcade,PC,80,7.3,Online
2321,2364,Battle Brothers,2017,OverhypeStudios,Strategy;Turn-Based;Tactics,PC,80,7.5,1 Player
2322,2365,BoxBoxBoy!,2016,HALLabs,Puzzle;Action,3DS,80,7.9,1 Player


In [ ]:
df_comentarios['Comment'][0] + ' ' + df_comentarios['Comment'][1]

"Everything in OoT is so near at perfection, its really no wonder that this game has such a huge fanbase and is so highly rated among players and critics. I won't bore you with what everyone is already saying about how amazing this game is.  But if you're on the fence about whether to give this classic a try, go read IGN's review."

In [ ]:
#creo una función que me devuelva el string concatenado de todos los comentarios de un juego en particular
#recibe el df y el título del juego y devuelve el párrafo resultante

def obtenerParrafo(juego):

  parrafo = ''
  for comment in df_comentarios[df_comentarios['Title'] == juego]['Comment']:
    parrafo += ' ' + comment
  
  return parrafo

In [ ]:
df_juegos['Paragraph'] = df_juegos['Title'].apply(obtenerParrafo)

In [ ]:
df_juegos = df_juegos[['Title', 'Year', 'Publisher', 'Genre', 'Platform', 'Metascore',
       'Avg_Userscore', 'No_Players', 'Paragraph']]

In [ ]:
df_juegos

,Title,Year,Publisher,Genre,Platform,Metascore,Avg_Userscore,No_Players,Paragraph
0,The Legend of Zelda: Ocarina of Time,1998,Nintendo,Action Adventure;Fantasy,Nintendo64,99,9.1,1 Player,"Everything in OoT is so near at perfection, i..."
1,Tony Hawk's Pro Skater 2,2000,NeversoftEntertainment,Sports;Alternative;Skateboarding,PlayStation,98,7.4,Multiplayer,"Breath-taking, still replayable to this day...."
2,Grand Theft Auto IV,2008,RockstarNorth,Action Adventure;Modern;Modern;Open-World,PlayStation3,98,7.5,1 Player,"After playing GTA III, Vice City and San Andr..."
3,SoulCalibur,1999,Namco,Action;Fighting;3D,Dreamcast,98,8.6,Multiplayer,The best fighting game I've ever played by f...
4,Super Mario Galaxy,2007,Nintendo,Action;Platformer;Platformer;3D;3D,Wii,97,9.0,1 Player,Nintendo always impresses me. Mario Galaxy wa...
...,...,...,...,...,...,...,...,...,...
2319,N++: Ultimate Edition,2018,MetanetSoftwareInc.,Action;Platformer;2D,Switch,80,7.4,1 Player,Platform game and action very entertaining an...
2320,"Guns, Gore & Cannoli 2",2018,CrazyMonkeyStudios,Action;Shooter;Third-Person;Arcade,PC,80,7.3,Online,"Great artwork, but terrible, terrible, contro..."
2321,Battle Brothers,2017,OverhypeStudios,Strategy;Turn-Based;Tactics,PC,80,7.5,1 Player,A very good game but not for casual gamers. Y...
2322,BoxBoxBoy!,2016,HALLabs,Puzzle;Action,3DS,80,7.9,1 Player,Pros:-$4.99 guys. Four dollars and ninety-nin...


# Modelo dado título

Acá limpio textos con stopwords

In [ ]:
# Lista de stopwords
stopwords = nltk.corpus.stopwords.words('english')

# Creamos el objeto contador de palabras, pidiéndole que remueve
# las stopwords, los términos que aparecen en un único documento (min_df)
# y los términos que aparecen en más del 70% de los documentos (max_df).
# Esto es para eliminar palabras raras (o errores de tipeo) y 
# términos que seguramente son stopwords no incluídos en la lista
count = CountVectorizer(min_df = 2, max_df = 0.70, stop_words = stopwords)

# Ajustamos con los datos. Acá especificamente creamos una matriz documentos-términos
x_count = count.fit_transform(df_juegos['Paragraph'])

# Dimensions de la matriz doc-tér
print(x_count.shape)

(2324, 75643)


In [ ]:
# Creamos el objeto tf-idf. Le decimos además que devuelva los
# vectores documento con norma euclídea igual a 1 (norm = 'l2')
tfidf = TfidfTransformer(norm = 'l2')

# Creamos la matriz tf-idf a partir de la matriz de frecuencias
x_tfidf = tfidf.fit_transform(x_count)

In [ ]:
x_tfidf.shape

(2324, 75643)

In [ ]:
# Dimensión del espacio reducido 
n_components = 300

# Creación del objeto SVD
svd = TruncatedSVD(n_components = n_components)

In [ ]:
# Ajuste con los datos y descripción en el espacio reducido
# Trabajamos con la matriz traspuesta
x_svd = svd.fit_transform(x_tfidf.T)

# Dimensión de la matriz de datos en el espacio reducido
print(x_svd.shape)

(75643, 300)


In [ ]:
# Identifiquemos el índice de alguna palabra de referencia
index_word = count.vocabulary_['driving']

In [ ]:
# Diccionario índice: término
vocabulary = {item: key for key, item in count.vocabulary_.items()}

In [ ]:
from scipy.spatial.distance import cosine as cosine_distance

# Calculo las distancia coseno del término de referencia respecto de los otros
similarities = [1.00 - cosine_distance(x_svd[index_word], x_svd[i]) for i in range(x_svd.shape[0])]

In [ ]:
import heapq

# Encuentra las 10 similaridades más grande
max_matching = heapq.nlargest(10, similarities)

# Encuentra los índices correspondiente a las similaridades más altas
max_matching_index = [np.where(similarities == m)[0][0] for m in max_matching]
#To decide on a suitable number of topics, you can compare the goodness-of-fit of LDA models fit with varying numbers of topics. You can evaluate the goodness-of-fit of an LDA model by calculating the perplexity of a held-out set of documents. T
# Devuelve los términos correspondientes a cada índice
for i in max_matching_index:
  print(vocabulary[i])

driving
drive
car
steering
wheel
driver
cars
handling
handbrake
molesting


In [ ]:
df_juegos[df_juegos['Title'] == ' ']

,Title,Year,Publisher,Genre,Platform,Metascore,Avg_Userscore,No_Players,Paragraph


## correr esto

In [ ]:
# k: indice de juego para probar el modelo
k = np.random.randint(low = 0, high = len(df_juegos))
k = 1242
df_juegos['Title'][k]

'Halo 5: Guardians'

In [ ]:
# Buscamos el índice de un juego en particular
track_index = df_juegos[df_juegos['Title'] == df_juegos['Title'][k]].index[0]

In [ ]:
# Dimensiones de las componentes singulares
print(svd.components_.shape)

(300, 2324)


In [ ]:
# Calculo las distancia coseno de la canción respecto de todas las demás canciones
similarities = [1.00 - cosine_distance(svd.components_.T[track_index], svd.components_.T[i]) for i in range(svd.components_.shape[1])]

In [ ]:
# Buscamos las n similaridades más grandes
n = 5

max_matching = heapq.nlargest(n, similarities)

# Printemos las similaridades
print(max_matching)

# Encuentra los índices correspondiente a las similaridades más altas
max_matching_index = [np.where(similarities == m)[0][0] for m in max_matching]

# Imprimimos los datos de las canciones más similares
df_juegos.iloc[max_matching_index]

[1.0, 0.979826903750459, 0.9761496057436103, 0.9632697263148001, 0.9497185871010198]


,Title,Year,Publisher,Genre,Platform,Metascore,Avg_Userscore,No_Players,Paragraph
1242,Halo 5: Guardians,2015,343Industries,Shooter;Sci-Fi;Sci-Fi;Action;Shooter;First-Per...,XboxOne,84,6.5,Multiplayer,"This review contains spoilers, cl..."
614,Halo 4,2012,343Industries,Shooter;Sci-Fi;Sci-Fi;Action;Shooter;First-Per...,Xbox360,87,7.0,Multiplayer,halo 4....... were to begin. well lets start ...
233,Halo: Reach,2010,Bungie,Action;Shooter;Shooter;First-Person;Sci-Fi;Sci...,Xbox360,91,8.1,Multiplayer,This game delivered on every front. From the ...
62,Halo 3,2007,"BungieSoftware,Bungie",Action;Shooter;Shooter;First-Person;Sci-Fi;Sci...,Xbox360,94,7.9,Multiplayer,Anyone who ever gave this game anything less ...
1446,Halo 3: ODST,2009,"BungieSoftware,Bungie",Action;Shooter;Shooter;First-Person;Sci-Fi;Sci...,Xbox360,83,7.2,Online,Campaign 10/10 FireFight 10/10 its all aweso...


In [ ]:
df_juegos.iloc[max_matching_index][['Title', 'Genre']]

,Title,Genre
1242,Halo 5: Guardians,Shooter;Sci-Fi;Sci-Fi;Action;Shooter;First-Per...
614,Halo 4,Shooter;Sci-Fi;Sci-Fi;Action;Shooter;First-Per...
233,Halo: Reach,Action;Shooter;Shooter;First-Person;Sci-Fi;Sci...
62,Halo 3,Action;Shooter;Shooter;First-Person;Sci-Fi;Sci...
1446,Halo 3: ODST,Action;Shooter;Shooter;First-Person;Sci-Fi;Sci...


# Probando cosas: Modelo dado intereses
Dado un comentario (input), devolvemos los títulos más parecidos de acuerdo con la distancia coseno del comentario al resto

In [ ]:
comentario = input('Write down your interests: ')

Write down your interests: It is the scariest game ever, I could not sleep for weeks.


In [ ]:
df_1 = df_juegos.copy()

In [ ]:
dictionary = {'Title': 'New Commentary', 'Paragraph': comentario}

In [ ]:
df_1 = df_1.append(dictionary, ignore_index = True)

In [ ]:
df_1

,Title,Year,Publisher,Genre,Platform,Metascore,Avg_Userscore,No_Players,Paragraph
0,The Legend of Zelda: Ocarina of Time,1998,Nintendo,Action Adventure;Fantasy,Nintendo64,99,9.1,1 Player,"Everything in OoT is so near at perfection, i..."
1,Tony Hawk's Pro Skater 2,2000,NeversoftEntertainment,Sports;Alternative;Skateboarding,PlayStation,98,7.4,Multiplayer,"Breath-taking, still replayable to this day...."
2,Grand Theft Auto IV,2008,RockstarNorth,Action Adventure;Modern;Modern;Open-World,PlayStation3,98,7.5,1 Player,"After playing GTA III, Vice City and San Andr..."
3,SoulCalibur,1999,Namco,Action;Fighting;3D,Dreamcast,98,8.6,Multiplayer,The best fighting game I've ever played by f...
4,Super Mario Galaxy,2007,Nintendo,Action;Platformer;Platformer;3D;3D,Wii,97,9.0,1 Player,Nintendo always impresses me. Mario Galaxy wa...
...,...,...,...,...,...,...,...,...,...
2320,"Guns, Gore & Cannoli 2",2018,CrazyMonkeyStudios,Action;Shooter;Third-Person;Arcade,PC,80,7.3,Online,"Great artwork, but terrible, terrible, contro..."
2321,Battle Brothers,2017,OverhypeStudios,Strategy;Turn-Based;Tactics,PC,80,7.5,1 Player,A very good game but not for casual gamers. Y...
2322,BoxBoxBoy!,2016,HALLabs,Puzzle;Action,3DS,80,7.9,1 Player,Pros:-$4.99 guys. Four dollars and ninety-nin...
2323,Etrian Odyssey Untold: The Millennium Girl,2013,Atlus,Role-Playing;First-Person;First-Person;Western...,3DS,80,8.1,1 Player,A fantastic retelling of the original Etrian ...


In [ ]:
# Lista de stopwords
stopwords = nltk.corpus.stopwords.words('english')

# Creamos el objeto contador de palabras, pidiéndole que remueve
# las stopwords, los términos que aparecen en un único documento (min_df)
# y los términos que aparecen en más del 70% de los documentos (max_df).
# Esto es para eliminar palabras raras (o errores de tipeo) y 
# términos que seguramente son stopwords no incluídos en la lista
count = CountVectorizer(min_df = 2, max_df = 0.70, stop_words = stopwords)

# Ajustamos con los datos. Acá especificamente creamos una matriz documentos-términos
x_count = count.fit_transform(df_1['Paragraph'])

# Dimensions de la matriz doc-tér
print(x_count.shape)

(2325, 75644)


In [ ]:
# Creamos el objeto tf-idf. Le decimos además que devuelva los
# vectores documento con norma euclídea igual a 1 (norm = 'l2')
tfidf = TfidfTransformer(norm = 'l2')

# Creamos la matriz tf-idf a partir de la matriz de frecuencias
x_tfidf = tfidf.fit_transform(x_count)

In [ ]:
# Dimensión del espacio reducido 
n_components = 300

# Creación del objeto SVD
svd = TruncatedSVD(n_components = n_components)

In [ ]:
# Ajuste con los datos y descripción en el espacio reducido
# Trabajamos con la matriz traspuesta
x_svd = svd.fit_transform(x_tfidf.T)

# Dimensión de la matriz de datos en el espacio reducido
print(x_svd.shape)

(75644, 300)


In [ ]:
# Identifiquemos el índice de alguna palabra de referencia
index_word = count.vocabulary_['soccer']

In [ ]:
# Diccionario índice: término
vocabulary = {item: key for key, item in count.vocabulary_.items()}

In [ ]:
len(vocabulary)

75644

In [ ]:
from scipy.spatial.distance import cosine as cosine_distance

# Calculo las distancia coseno del término de referencia respecto de los otros
similarities = [1.00 - cosine_distance(x_svd[index_word], x_svd[i]) for i in range(x_svd.shape[0])]

In [ ]:
len(similarities)

75644

In [ ]:
import heapq

# Encuentra las 10 similaridades más grande
max_matching = heapq.nlargest(10, similarities)

# Encuentra los índices correspondiente a las similaridades más altas
max_matching_index = [np.where(similarities == m)[0][0] for m in max_matching]

# Devuelve los términos correspondientes a cada índice
for i in max_matching_index:
  print(vocabulary[i])

soccer
beckham
eleven
pes5
freekicks
magnets
muchhhh
we6
graphices
we7


In [ ]:
# k: indice de juego para probar el modelo
#k = np.random.randint(low = 0, high = len(df_juegos))
#df_juegos['Title'][k]

In [ ]:
# Buscamos el índice de un juego en particular
#track_index = df_1[df_1['Title'] == df_juegos['Title'][k]].index[0]

In [ ]:
# Dimensiones de las componentes singulares
print(svd.components_.shape)

(300, 2325)


In [ ]:
# Calculo las distancia coseno de la canción respecto de todas las demás canciones
similarities = [1.00 - cosine_distance(svd.components_.T[2324], svd.components_.T[i]) for i in range(svd.components_.shape[1])]

In [ ]:
# Buscamos las n similaridades más grandes
n = 6

max_matching = heapq.nlargest(n, similarities)

# Printemos las similaridades
print(max_matching)

# Encuentra los índices correspondiente a las similaridades más altas
max_matching_index = [np.where(similarities == m)[0][0] for m in max_matching[1:]]

# Imprimimos los datos de las canciones más similares
df_1.iloc[max_matching_index]

[1.0, 0.5182416207031073, 0.401190308598653, 0.37752556218494027, 0.33550305660902824, 0.30814222554470594]


,Title,Year,Publisher,Genre,Platform,Metascore,Avg_Userscore,No_Players,Paragraph
1194,Fatal Frame II: Crimson Butterfly Director's Cut,2004,Tecmo,Action Adventure;Horror,Xbox,84,8.4,1 Player,Warning: not for those with weak hearts. Th...
2270,Rocket Riot,2009,Codeglue,Action;General;General,Xbox360,80,7.6,Online,+ Fun to play. + Great looking game + Fun pa...
1441,Wakeboarding Unleashed Featuring Shaun Murray,2003,ShabaGames,Sports;Alternative;Wakeboarding,PlayStation2,83,8.3,Multiplayer,A unique co-op mode that lets you drive the ...
2167,Fatal Frame II: Crimson Butterfly,2003,Tecmo,Action Adventure;Horror,PlayStation2,81,9.0,1 Player,Fatal Frame II: Crimson Butterfly is and impr...
2244,Def Jam Vendetta,2003,AkiCorp.,Action;Fighting;Wrestling,GameCube,81,8.3,Multiplayer,This game is hot. It rocks hard and the gamep...


In [ ]:
comentario

'It is the scariest game ever, I could not sleep for weeks.'

In [ ]:
df_reviews = df_1[['Title', 'Paragraph']]

In [ ]:
df_reviews

,Title,Paragraph
0,The Legend of Zelda: Ocarina of Time,"Everything in OoT is so near at perfection, i..."
1,Tony Hawk's Pro Skater 2,"Breath-taking, still replayable to this day...."
2,Grand Theft Auto IV,"After playing GTA III, Vice City and San Andr..."
3,SoulCalibur,The best fighting game I've ever played by f...
4,Super Mario Galaxy,Nintendo always impresses me. Mario Galaxy wa...
...,...,...
2320,"Guns, Gore & Cannoli 2","Great artwork, but terrible, terrible, contro..."
2321,Battle Brothers,A very good game but not for casual gamers. Y...
2322,BoxBoxBoy!,Pros:-$4.99 guys. Four dollars and ninety-nin...
2323,Etrian Odyssey Untold: The Millennium Girl,A fantastic retelling of the original Etrian ...


# + Sentiment

Filtro sentiment

In [2]:
!pip install pysentimiento

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 312 kB 9.1 MB/s 
     |████████████████████████████████| 175 kB 54.3 MB/s 
     |████████████████████████████████| 4.4 MB 50.3 MB/s 
     |████████████████████████████████| 1.1 MB 42.9 MB/s 
     |████████████████████████████████| 212 kB 51.4 MB/s 
     |████████████████████████████████| 101 kB 13.3 MB/s 
     |████████████████████████████████| 140 kB 61.7 MB/s 
     |████████████████████████████████| 596 kB 69.6 MB/s 
     |████████████████████████████████| 127 kB 77.0 MB/s 
     |████████████████████████████████| 6.6 MB 38.2 MB/s 
     |████████████████████████████████| 94 kB 4.5 MB/s 
     |████████████████████████████████| 271 kB 70.1 MB/s 
     |████████████████████████████████| 144 kB 55.4 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=4348b37c104c352ce649669bc4af5a4a51e40d35a76416ab0853639c6eca3970
 

In [3]:
from pysentimiento import create_analyzer
analyzer = create_analyzer('sentiment', lang = 'en')

Downloading:   0%|          | 0.00/295 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/890 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/515M [00:00<?, ?B/s]

In [4]:
comentario = "Fantastic atmosphere, great graphics that stands the test of time, great storyline, intelligent puzzles and a lot of action make Half-Life 2 one of the best FPS games ever. I liked the classic first part too but this is the one that really made me a fan"
comentario1 = "Ubisoft disappointing us as always. It's just a bad GTA copy but with hackers and full of bugs"
comentario2 = "I had nightmares after playing it. No game will ever be as scary as this one. Best horror game ever"
comentario3 = "I had nightmares after playing it. No game will ever be as scary as this one."
X = analyzer.predict(comentario)
X1 = analyzer.predict(comentario1)
X2 = analyzer.predict(comentario2)
X3 = analyzer.predict(comentario3)

In [5]:
print(X)
print(X1)
print(X2)
print(X3)

AnalyzerOutput(output=POS, probas={POS: 0.992, NEU: 0.007, NEG: 0.001})
AnalyzerOutput(output=NEG, probas={NEG: 0.985, NEU: 0.009, POS: 0.006})
AnalyzerOutput(output=POS, probas={POS: 0.589, NEU: 0.241, NEG: 0.171})
AnalyzerOutput(output=NEG, probas={NEG: 0.983, NEU: 0.012, POS: 0.004})


In [ ]:

#if X.output == 'NEG':
#  print('No recomiendo')